# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [4]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head(20)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [9]:
quantitydf=data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})

quantitydf.head(110)

#data.groupby('month', as_index=False).agg({"duration": "sum"})
#data.groupby(['month', 'network_type'])['date'].count()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
200        Quiche Assorted                       1
           Rambutan                              1
           Remy Red                              2
           Salmon - Atlantic, Skin On            1
           Sauce - Gravy, Au Jus, Mix            1

[110 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [37]:
quantitydf.pivot_table(index="ProductName",values="Quantity",columns="CustomerID",fill_value=0)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [53]:
pivot=quantitydf.pivot_table(index="ProductName",values="Quantity",columns="CustomerID",fill_value=0)

In [54]:
from scipy.spatial.distance import pdist, squareform

In [55]:
distances = pdist(pivot.T, 'euclidean')

In [56]:

print(squareform(distances))

[[  0.          11.91637529  10.48808848 ... 228.62851966 239.
  229.77380181]
 [ 11.91637529   0.          11.74734012 ... 228.01096465 239.03765394
  229.70415756]
 [ 10.48808848  11.74734012   0.         ... 228.08112592 238.26665734
  229.77380181]
 ...
 [228.62851966 228.01096465 228.08112592 ...   0.         304.13812651
  305.16389039]
 [239.         239.03765394 238.26665734 ... 304.13812651   0.
  303.10889132]
 [229.77380181 229.70415756 229.77380181 ... 305.16389039 303.10889132
    0.        ]]


In [57]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [59]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [78]:
new=quantitydf.loc[[264,3535,3317,2503,3305]]


Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

In [79]:
new.loc[[264]]

Quantity
CustomerID ProductName                            
264        Apricots - Halves                     1
           Apricots Fresh                        1
           Bacardi Breezer - Tropical            1
           Bagel - Plain                         1
           Banana - Leaves                       1
...                                            ...
           Wine - Chardonnay South               1
           Wine - Gato Negro Cabernet            1
           Wine - Redchard Merritt               1
           Wine - Sogrape Mateus Rose            1
           Wine - Wyndham Estate Bin 777         1

[62 rows x 1 columns]

In [80]:
 new.loc[[3535]]

Quantity
CustomerID ProductName                              
3535       Anchovy Paste - 56 G Tube               1
           Apricots - Dried                        1
           Bacardi Breezer - Tropical              1
           Bandage - Fexible 1x3                   1
           Beef - Chuck, Boneless                  1
           Beef - Short Loin                       1
           Beef - Top Sirloin - Aaa                1
           Beef Wellington                         1
           Cheese - Cambozola                      1
           Cheese Cloth No 100                     1
           Chicken - Soup Base                     1
           Chinese Foods - Chicken                 1
           Cod - Black Whole Fillet                1
           Cornflakes                              1
           Curry Paste - Madras                    1
           Dc Hikiage Hira Huba                    1
           Dried Figs                              1
           Eggplant - Asian                        1
           Fish - Scallops, Cold Smoked            1
           Foam Dinner Plate                       1
           Fondant - Icing                         1
           Garbage Bags - Clear                    1
           Garlic - Elephant                       1
           Halibut - Steaks                        1
           Jagermeister                            1
           Juice - Orange                          1
           Ketchup - Tomato                        1
           Lamb - Ground                           1
           Lamb - Pieces, Diced                    1
           Milk - 2%                               1
           Milk Powder                             1
           Muffin Batt - Blueberry Passion         1
           Ocean Spray - Kiwi Strawberry           1
           Ocean Spray - Ruby Red                  1
           Pasta - Orecchiette                     1
           Pepper - Black, Whole                   1
           Sausage - Breakfast                     1
           Scallops - 10/20                        1
           Scallops 60/80 Iqf                      1
           Sea Bass - Whole                        1
           Sherry - Dry                            1
           Sole - Dover, Whole, Fresh              1
           Soup - Campbells Bean Medley            1
           Soupfoamcont12oz 112con                 1
           Steam Pan - Half Size Deep              1
           Sugar - Fine                            1
           Table Cloth 54x72 White                 1
           Tahini Paste                            1
           Tofu - Firm                             1
           Towels - Paper / Kraft                  1
           Tray - 16in Rnd Blk                     1
           Wanton Wrap                             1
           Wine - Blue Nun Qualitatswein           1
           Wine - Magnotta, Merlot Sr Vqa          1
           Yoghurt Tubes                           1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [81]:
newranked=new.sort_values(by='Quantity', ascending=False)
#df.sort_values(by='col1', ascending=False)

In [82]:
newranked

Quantity
CustomerID ProductName                                
2503       Butter - Unsalted                         2
           Bar - Granola Trail Mix Fruit Nut         2
3305       Wine - Ej Gallo Sierra Valley             2
2503       Knife Plastic - White                     2
264        Guinea Fowl                               2
...                                                ...
2503       Soup - Campbells, Cream Of                1
           Soup - Canadian Pea, Dry Mix              1
           Tea - Decaf Lipton                        1
           Tia Maria                                 1
3535       Yoghurt Tubes                             1

[270 rows x 1 columns]

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.